# This is a Jupyter Notebook for the Applied Data Science Capstone Project

In [3]:
import pandas as pd
import numpy as np

# Segmenting and Clustering Neighborhoods in Toronto

## Part 1: Web Scraping

In [4]:
from bs4 import BeautifulSoup
import requests

In [3]:
url = "https://www.yelp.com/collection/Z3iwKFIDwScWaYjcBG6v-g?utm_content=Collections&utm_source=ishare"

In [208]:
url = "https://www.yelp.com/collection/Z3iwKFIDwScWaYjcBG6v-g?sort_by=distance"

In [221]:
page = requests.get(url)

In [4]:
import os, sys

if sys.version_info.major < 3:
    from urllib import url2pathname
else:
    from urllib.request import url2pathname

class LocalFileAdapter(requests.adapters.BaseAdapter):
    """Protocol Adapter to allow Requests to GET file:// URLs

    @todo: Properly handle non-empty hostname portions.
    """

    @staticmethod
    def _chkpath(method, path):
        """Return an HTTP status for the given filesystem path."""
        if method.lower() in ('put', 'delete'):
            return 501, "Not Implemented"  # TODO
        elif method.lower() not in ('get', 'head'):
            return 405, "Method Not Allowed"
        elif os.path.isdir(path):
            return 400, "Path Not A File"
        elif not os.path.isfile(path):
            return 404, "File Not Found"
        elif not os.access(path, os.R_OK):
            return 403, "Access Denied"
        else:
            return 200, "OK"

    def send(self, req, **kwargs):  # pylint: disable=unused-argument
        """Return the file specified by the given request

        @type req: C{PreparedRequest}
        @todo: Should I bother filling `response.headers` and processing
               If-Modified-Since and friends using `os.stat`?
        """
        path = os.path.normcase(os.path.normpath(url2pathname(req.path_url)))
        response = requests.Response()

        response.status_code, response.reason = self._chkpath(req.method, path)
        if response.status_code == 200 and req.method.lower() != 'head':
            try:
                response.raw = open(path, 'rb')
            except (OSError, IOError) as err:
                response.status_code = 500
                response.reason = str(err)

        if isinstance(req.url, bytes):
            response.url = req.url.decode('utf-8')
        else:
            response.url = req.url

        response.request = req
        response.connection = self

        return response

    def close(self):
        pass

In [5]:
requests_session = requests.session()
requests_session.mount('file://', LocalFileAdapter())
r = requests_session.get('file:///C:/Users/mattw/Downloads/full_clt.html')

In [6]:
soup = BeautifulSoup(r.text)

In [7]:
r_list = soup.findAll(class_ = 'biz-name js-analytics-click')
r2_list = []

In [8]:
for z in range(1,len(r_list)):
    start = str(r_list[z]).find('<span>') + 6
    end = str(r_list[z]).find('</span>')
    name = str(r_list[z])[start:(end-1)]
    #print(name)
    r2_list.append(name)

In [9]:
r2_list

['Cabo Fish Tac',
 'Hazelnuts Creperi',
 'Fenwick’',
 'Cantina Do Brasi',
 'TRUE Crafted Pizz',
 'Birdsong Brewin',
 'The Suffolk Punc',
 'The Waterman Fish Ba',
 'Bonzai Thai &amp; Japanese Cuisin',
 'Price’s Chicken Coo',
 'Big Ben British Restaurant &amp; Pu',
 'Chatpata',
 'Chatpata',
 'Crispy Banh M',
 'Crispy Banh M',
 'Zio Casual Italia',
 'Bisonte Pizz',
 'Bisonte Pizz',
 'Zen Asian Fusio',
 'Queen City Craft and Gourme',
 'Ari',
 'Banh Mi Brother',
 'Choi’s Korea &amp; Win',
 'VeloPop',
 'Eggs Up Gril',
 'Angeline’',
 'Murphy’s Kitchen &amp; Ta',
 'Inishmor',
 'Moosehead Gril',
 'DD Pecker',
 'Fonta Flora Brewery - Optimist Hal',
 'Mr. K’s Soft Ice Cream &amp; Drive I',
 'Three Amigos Mexican Restaurant and Cantin',
 'Two Scoops Creamer',
 'Two Scoops Creamer',
 'Town Brewing Compan',
 'Veltre',
 'Viva Chicke',
 'Viva Chicken - Gastoni',
 'Viva Chicke',
 'Viva Chicken RedSton',
 'Viva Chicken of Northcros',
 'Viva Chicke',
 'Viva Chicke',
 'Viva Chicke',
 'Viva Chicke',
 'Whis

In [10]:
# Empty list
venue_list = []

In [11]:
def get_clt_restaurants(name):
    
    CLIENT_ID = 'XHZAVLMJZPZ1YERWSOQP0RZSG1JTQV24LLJOC3CHJZBFUCWM'
    CLIENT_SECRET = 'FKWRVOLJYEBTMABNFPMRYRJ3JOQLKTDE2JPSZKUGJYHQSAMS'
    VERSION = "20180323"
    
    # create the API request URL
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&near={}&query={}'.format(
           CLIENT_ID, 
           CLIENT_SECRET, 
           VERSION, 
           'Charlotte, NC',
           name)
    
    # getting response from query
    resp = requests.get(url=url).json()["response"]['groups'][0]['items'][0]['venue']
    
    # getting website url
    id_url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(
           resp['id'],
           CLIENT_ID, 
           CLIENT_SECRET, 
           VERSION)
    #print(id_url)
    
    
    # Get website name
    venue_info = requests.get(url=id_url).json()['response']['venue']
    
    # Check if there is a website, if not, get delivery - or return nothing
    if 'url' in venue_info:
        website = venue_info['url']
    elif 'delivery' in venue_info:
        website = venue_info['delivery']['url']
    else: website = 'no_url' 
    
    #print(website)
    
    # return only relevant information for each nearby venue
    print(resp['name'])
    venue_list.append([(resp['name'], 
                    resp['id'],
                   resp['location']['formattedAddress'],
                   resp['location']['lat'],
                   resp['location']['lng'],
                   resp['categories'][0]['shortName'],
                   website)])

In [12]:
# loop over restaurants
for ii in r2_list:
    get_clt_restaurants(ii)

Cabo Fish Taco
Hazelnuts Creperie
Fenwicks Restaurant
Three Amigos Mexican Grill and Cantina
TRUE Crafted Pizza
Catawba Brewing Charlotte
The Fillmore Charlotte
The Fillmore Charlotte
Bahn Thai
Viva Chicken Park Road
VBGB Beer Hall & Garden/Restaurant
Chatpatay
Chatpatay
Bánh Mì Brothers
Bánh Mì Brothers
Rita's Italian Ice & Frozen Custard
Benny Pennello's Pizza
Benny Pennello's Pizza
Zen Massage Charlotte
Craft Tasting Room and Growler Shop
Aria Tuscan Grill
Bánh Mì Brothers
Dae Bak Korean Restaurant
Velo Pops
Lego pop-up shop
Angeline's
Stacks Kitchen
Inishmore
Little Village Grill
DD Peckers Wing Shop
Birdsong Brewing Co.
Tony's Ice Cream
VBGB Beer Hall & Garden/Restaurant
Hub City Scoops
Hub City Scoops
Catawba Brewing Charlotte
Veltree Vegan Soul Food And Juice Bar
Viva Chicken Park Road
Viva Chicken Park Road
Viva Chicken Park Road
Viva Chicken Park Road
Bank of America Stadium
Viva Chicken Park Road
Viva Chicken Park Road
Viva Chicken Park Road
Viva Chicken Park Road
Bakersfield

KeyError: 'venue'

In [15]:
len(venue_list)

84

In [16]:
# getting names
names = []
for r in venue_list:
    names.append(r[0][0])

In [17]:
# getting foursquare ID
fid = []
for r in venue_list:
    fid.append(r[0][1])

In [18]:
# getting formatted address
address = []
for r in venue_list:
    address.append(r[0][2][0])

In [19]:
# getting latitude
lat = []
for r in venue_list:
    lat.append(r[0][3])

In [20]:
# getting longitude
lon = []
for r in venue_list:
    lon.append(r[0][4])

In [21]:
# getting categories
cat = []
for r in venue_list:
    cat.append(r[0][5])

In [22]:
# getting website
web = []
for r in venue_list:
    web.append(r[0][6])

In [25]:
venues = pd.DataFrame({'Name':names, 'Foursquare_ID':fid, 'Address': address, 'Latitude': lat, 'Longitude':lon, 'Category':cat, 'Website':web})

In [205]:
venues = venues.drop_duplicates()

In [26]:
venues.to_excel("C:/Users/mattw/Desktop/Charlotte_Restaurants2.xlsx")  

In [45]:
len(r2_list)

29

In [29]:
for jj in range(r2_list):
    r = r2_list[jj]  

,Restaurant,FsID,Address,Type


In [142]:
#id_url = "https://api.foursquare.com/v2/venues/4b05863ff964a520165a22e3"

# create the API request URL
id_url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(
           '4b05863ef964a520025a22e3',
           CLIENT_ID, 
           CLIENT_SECRET, 
           VERSION)

In [143]:
id_resp = requests.get(url=id_url).json()

In [144]:
if 'url' in id_resp['response']['venue']:
        w = id_resp['response']['venue']['url']
elif 'delivery' in id_resp['response']['venue']:
        w = id_resp['response']['venue']['delivery']['url']
else: w = 'no_url'

In [124]:
resp = requests.get(url=url).json()["response"]['groups'][0]['items']

In [89]:
resp

[{'reasons': {'count': 0,
   'items': [{'summary': 'This spot is popular',
     'type': 'general',
     'reasonName': 'globalInteractionReason'}]},
  'venue': {'id': '4b05863ff964a520165a22e3',
   'name': 'Fenwicks Restaurant',
   'contact': {},
   'location': {'address': '511 Providence Rd',
    'lat': 35.20314967930705,
    'lng': -80.82449160336581,
    'labeledLatLngs': [{'label': 'display',
      'lat': 35.20314967930705,
      'lng': -80.82449160336581}],
    'postalCode': '28207',
    'cc': 'US',
    'city': 'Charlotte',
    'state': 'NC',
    'country': 'United States',
    'formattedAddress': ['511 Providence Rd',
     'Charlotte, NC 28207',
     'United States']},
   'categories': [{'id': '4bf58dd8d48988d14e941735',
     'name': 'American Restaurant',
     'pluralName': 'American Restaurants',
     'shortName': 'American',
     'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/default_',
      'suffix': '.png'},
     'primary': True}],
   'verified': False,
   '

In [64]:
resp['name']
resp['location']
resp['categories'][0]['shortName']

'American'

In [74]:
venue_list = []

## Part 2: Locations

## Part 3: Mapping

In [243]:
import folium
from geopy.geocoders import Nominatim

### Subsetting to just the Toronto Boroughs

In [244]:
# Subsetting to just the Toronto Boroughs
df4 = df3[df3['Borough'].str.contains("Toronto")].drop(70)

### Getting Toronto Location

### Defining Function to get values for multiple boroughs

### CLT Agenda

The returned some Venue Categories of **Venue Categeory**, so I am just removing for now. Out of all entries, it was only n = 4.

In [1]:
cltagn = "https://www.charlotteagenda.com/199599/charlotte-restaurants-that-are-offering-curbside-pickup-delivery-and-discounts/"

In [6]:
# Parsing page from requests
clt_page = requests.get(cltagn)
clt_text = BeautifulSoup(clt_page.text)

In [17]:
ag_list = clt_text.findAll('h2')

In [28]:
len(ag_list)

196

In [23]:
websites = []
name = []

for z in range(1,len(ag_list)):
    
    # Getting websites
    web_start = str(ag_list[z]).find('<a href="') + 9
    web_end = str(ag_list[z]).find('" rel')
    web_name = str(ag_list[z])[web_start:(web_end-1)]
    websites.append(web_name)
    
    # Getting Name
    name_start = str(ag_list[z]).find('_blank">') + 8
    name_end = str(ag_list[z]).find('</a>')
    name_restaurant = str(ag_list[z])[name_start:(name_end)]
    name.append(name_restaurant)
    
    print("The restaurant name is {} and the website is {}".format(name_restaurant, web_name))

The restaurant name is Ace No. 3 and the website is https://www.aceno3.com
The restaurant name is Alexander Michael’s and the website is https://almikestavern.com
The restaurant name is Aliño Pizzeria and The Barcelona Burger and Beer Garden and the website is https://barcelonaburger.com
The restaurant name is All-American Pub and the website is https://www.allamericanpubclt.com
The restaurant name is Amelie’s French Cafe &amp; Bakery and the website is https://ameliesfrenchbakery.com
The restaurant name is Anderson’s Catering and the website is https://andersonsdelivery.com
The restaurant name is Angeline’s and the website is https://www.angelinescharlotte.co
The restaurant name is Anntony’s Caribbean Cafe and the website is https://www.anntonys.com
The restaurant name is Aria Tuscan Grill and the website is https://www.ariacharlotte.com
The restaurant name is The Asbury and the website is https://theasbury.com
The restaurant name is Bad Daddy’s and the website is https://www.baddaddy

In [48]:
clt_agenda = pd.DataFrame({"Name" : name, "Website": websites})

In [49]:
clt_agenda

,Name,Website
0,Ace No. 3,https://www.aceno3.com
1,Alexander Michael’s,https://almikestavern.com
2,Aliño Pizzeria and The Barcelona Burger and Be...,https://barcelonaburger.com
3,All-American Pub,https://www.allamericanpubclt.com
4,Amelie’s French Cafe &amp; Bakery,https://ameliesfrenchbakery.com
...,...,...
190,Yamazaru Sushi &amp; Sake,http://www.yamazarusushisake.com
191,Your Mom’s Donuts,http://www.yourmomsdonutsnc.com
192,Zen Asian Fusion,https://www.zenasianfusion.com
193,Zia Pia imports + Italian Kitchen,https://ziapia.com


In [25]:
ag_list_par = clt_text.findAll('p')

In [80]:
para = []

for z in range(1,len(ag_list_par)):
    
    # Getting websites
    start = str(ag_list_par[z]).find('<p><em>') + 4
    end = str(ag_list_par[z]).find('</p>')
    para_name = str(ag_list_par[z])[start:(end-1)]
    para.append(para_name)
    
    #print("Paragraph {}: {}".format(z, para_name))

In [64]:
# Legion has three paragraphs, so need to concatenate
para[98] = para[98] + ". " + para[99] + ". " + para[100]

In [85]:
# North Italia
para[133] = para[133] + ". " + para[134]

'Offering take-out and delivery on DoorDash. The brewery is doing a drive-thru in the North Tryon parking lot from 4-7:30 p.m. Monday-Thursday. An update on weekend hours will be coming shortly. To place a take-out order at North Italia’s Charlotte location, call 980-279-8900. Adjusted hours are Monday through Friday 11 a.m. to 9 p.m., Saturday 12 p.m. to 10 p.m., and Sunday 12 p.m. to 9 p.m. To place a take-out order at North Italia’s Charlotte location, call 980-279-8900. Adjusted hours are Monday through Friday 11 a.m. to 9 p.m., Saturday 12 p.m. to 10 p.m., and Sunday 12 p.m. to 9 p.m'

In [91]:
# OMB
para[137] = para[137] + ". " + para[138] + ". " + para[139]

In [103]:
# Portfolinos
para[148] = para[148] + ". " + para[149] + ". " + para[150] + ". " + para[151] + ". " + para[152]

In [110]:
# Queen City Grounds
para[156] = para[156] + ". " + para[157]

In [114]:
# Deleting copied
to_remove = [99,100,134,138,139,149,150,151,152,157]
para2 = [v for i, v in enumerate(para) if i not in to_remove]

In [116]:
para2

['em>We will continue to update this list as information comes in. If we missed a restaurant that you’d like to see on the list, email us at <a href="mailto:hi@charlotteagenda.com" rel="noopener noreferrer" target="_blank">hi@charlotteagenda.com</a>.</em',
 'Order a burger and a shake to-go through Ace’s <a href="https://www.aceno3.com/" rel="noopener noreferrer" target="_blank">website</a> (preferred) or in-store',
 'Offering take-out Tuesday – Saturday from 11:00 a.m. – 2:00 p.m. and again from 5:00 p.m.-8:30 p.m',
 'Offering their three most popular dishes at up to 50% off or take-out and curbside pick-up.\xa0 The restaurants’ full menus will be available for take-out and curbside service, with prices cut on the most popular items',
 'Offering their full menu daily from 11 a.m.- 8 p.m. for take-out. Order<a href="https://www.allamericanpubclt.com/" rel="noopener noreferrer" target="_blank"> online </a>or call (704) 370-2000',
 'All five locations in the Charlotte region (this includ

In [117]:
clt_agenda['Information'] = para2[1:-2]

In [119]:
clt_agenda

,Name,Website,Information
0,Ace No. 3,https://www.aceno3.com,Order a burger and a shake to-go through Ace’s...
1,Alexander Michael’s,https://almikestavern.com,Offering take-out Tuesday – Saturday from 11:0...
2,Aliño Pizzeria and The Barcelona Burger and Be...,https://barcelonaburger.com,Offering their three most popular dishes at up...
3,All-American Pub,https://www.allamericanpubclt.com,Offering their full menu daily from 11 a.m.- 8...
4,Amelie’s French Cafe &amp; Bakery,https://ameliesfrenchbakery.com,All five locations in the Charlotte region (th...
...,...,...,...
190,Yamazaru Sushi &amp; Sake,http://www.yamazarusushisake.com,Offering curbside pick-up from 11:30 a.m. – 9 p.m
191,Your Mom’s Donuts,http://www.yourmomsdonutsnc.com,Offering curbside takeout
192,Zen Asian Fusion,https://www.zenasianfusion.com,Offering curbside pickup. Buy a $100 gift cert...
193,Zia Pia imports + Italian Kitchen,https://ziapia.com,Free delivery on groceries. Look for deli item...


In [120]:
clt_agenda.to_excel("C:/Users/mattw/Desktop/Charlotte_Restaurants3.xlsx")  

### Doing K Means Clustering Now

### Adding clustering labels onto the results from K-Means

In [255]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df4

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on = 'Neighbourhood')

toronto_merged.head() # check the last columns!

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,M4E,East Toronto,The Beaches,43.676357,-79.293031,1,Park,Trail,Health Food Store,Pub,Department Store,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop
41,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,0,Greek Restaurant,Coffee Shop,Italian Restaurant,Ice Cream Shop,Furniture / Home Store,Yoga Studio,Fruit & Vegetable Store,Restaurant,Pub,Pizza Place
42,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572,0,Park,Board Shop,Steakhouse,Sushi Restaurant,Ice Cream Shop,Brewery,Pub,Liquor Store,Fast Food Restaurant,Italian Restaurant
43,M4M,East Toronto,Studio District,43.659526,-79.340923,0,Café,Coffee Shop,Gastropub,Brewery,Bakery,Italian Restaurant,American Restaurant,Park,Seafood Restaurant,Sandwich Place
44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,4,Park,Swim School,Bus Line,Yoga Studio,Dim Sum Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant


### Vizualizing the output on a map

In [264]:
import matplotlib.cm as cm
import matplotlib.colors as colors

In [258]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_toronto)
       
map_toronto

I tried a number of different values for K, but nothing really resulted in greater distribution. It may be more fruitful to look at Greater Toronto as a whole, but it 

### Results if map does not render

In [3]:
%pylab inline
#import matplotlib.pyplot as plt
#import matplotlib.image as mpimg
#img=mpimg.imread('toronto_map.png')
#imgplot = plt.imshow(img, size=(8, 6))
#plt.show()

Populating the interactive namespace from numpy and matplotlib
